##1

A) In RSA Given n= 221 and e=5, find d.

In [ ]:
import math

def is_prime(num):
    """Check if a number is prime."""
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True



def find_prime_factors(n):
  """Find two prime factors of n"""
  for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            p = i
            q = n // i
            return p, q
  return None, None


# Function to find modular inverse of e mod phi_n
def modular_inverse(e, phi):
  return pow(e, -1, phi)

In [ ]:
n = 12091
e = 13

p, q = find_prime_factors(n)

if p and q:
    print(f"Found prime factors: p = {p}, q = {q}")
    # Calculate φ(n)
    phi_n = (p - 1) * (q - 1)
    print(f"Phi_n= {phi_n}")
    d = modular_inverse(e, phi_n)
    print(f"d = {d}")
elif is_prime(n):
    print(f"{n} is a prime number.")
    # If n is a prime number, φ(n) = n - 1
    phi_n = n - 1
    d = modular_inverse(e, phi_n)
    print(f"φ(n) = {phi_n}, d = {d}")
else:
    print("Prime factors not found.")

Found prime factors: p = 107, q = 113
Phi_n= 11872
d = 3653


B)
In RSA, encrypt the message “HOW ARE YOU” using 00 to 25 for letters A to Z and 26 for the space. Use different blocks to make P < n.


In [ ]:
import string

# Set up map for 0=a, 1=b....25=z, 26 = ' '
alphabet_map = {letter: index for index, letter in enumerate(string.ascii_lowercase)}
alphabet_map[' '] = 26
# Reverse map to decrypt
alphabet_reverse_map = {index: letter for letter, index in alphabet_map.items()}

In [ ]:
message = "crypto"

In [ ]:
coded_message = []

for char in message:
    code = alphabet_map[char]
    coded_message.append(code)

print(coded_message)

ciphertext = [(code**e) % n for code in coded_message]

print("Ciphertext: ", ciphertext)

[2, 17, 24, 15, 19, 14]
Ciphertext:  [8192, 9245, 2344, 7134, 10358, 2946]


In [ ]:
c1 = 8192**d % n
c2 = 9245**d % n

print(c1, c2)

2 17


##2

In ElGamal, given the prime p = 31:

a. Choose an appropriate e1 and d, then calculate e2.

b. Encrypt the message “HELLO”; use 00 to 25 for encoding. Use different blocks to make P < p.

c. Decrypt the ciphertext to obtain the plaintext.



In [ ]:
import random

message = "hello"


def elgamal_encrypt(block, e1, e2, p):
    r = random.randint(1, p - 2)      # Choose a random r < p
    c1 = pow(e1, r, p)                # c1 = e1^r % p
    c2 = (block * pow(e2, r)) % p     # c2 = (P * e2^r) % p
    return c1, c2


def elgamal_decrypt(c1, c2, d, p):
    s = pow(c1, d, p)              # s = c1^d % p
    s_inv = pow(s, p - 2, p)       # Modular inverse of s under modulo p
    plaintext = (c2 * s_inv) % p   # P = (c2 * s^-1) % p
    return plaintext

In [ ]:
p = 31
e1 = 3
d = 2               # Private key
e2 = pow(e1, d, p)  # Public key calculated as e2 = e1^d % p

print(f"Public Key: (p = {p}, e1 = {e1}, e2 = {e2})")
print(f"Private Key: d = {d}")

encoded_message = [alphabet_map[char] for char in message]

print(f"Encoded message: {encoded_message}")


ciphertext = [elgamal_encrypt(block, e1, e2, p) for block in encoded_message]

print("Ciphertext (c1, c2) pairs:")
for c in ciphertext:
    print(c)

Public Key: (p = 31, e1 = 3, e2 = 9)
Private Key: d = 2
Encoded message: [7, 4, 11, 11, 14]
Ciphertext (c1, c2) pairs:
(10, 18)
(20, 19)
(12, 3)
(6, 24)
(15, 19)


In [ ]:
decrypted_message = [elgamal_decrypt(c1, c2, d, p) for c1, c2 in ciphertext]

# Convert back to letters
decoded_message = ''.join(alphabet_reverse_map[block] for block in decrypted_message)

print(f"Decrypted message: {decoded_message}")


Decrypted message: hello


##3
An elliptic curve over E67(2, 3) as the elliptic curve (y^2 = x^3 + 2x + 3)
Bob selects e1 = (2, 22) and d = 4.


Alice wants to send the plaintext P = (24, 26) to Bob. She selects r = 2.


Write an algorithm to add two points for Alice to produce the ciphertext to send to Bob, and, algorithm for Bob to decrypt the message.

In [ ]:
def mod_inverse(a, p):
    """Calculate the modular inverse of a under modulo p."""
    return pow(a, p - 2, p)

def point_addition(P, Q, a, p):
    """Add two points P and Q on an elliptic curve y^2 = x^3 + ax + b over F_p."""
    if P == Q:
        # Point doubling
        numerator = (3 * P[0]**2 + a) % p
        denominator = (2 * P[1]) % p
    else:
        # Point addition
        numerator = (Q[1] - P[1]) % p
        denominator = (Q[0] - P[0]) % p

    if denominator == 0:
        return None  # Points are additive inverses

    m = (numerator * mod_inverse(denominator, p)) % p
    x3 = (m**2 - P[0] - Q[0]) % p
    y3 = (m * (P[0] - x3) - P[1]) % p

    return (x3, y3)

def scalar_multiplication(k, P, a, p):
    """Multiply point P by scalar k on elliptic curve over F_p."""
    R = None
    Q = P

    while k:
        if k & 1:
            R = point_addition(R, Q, a, p) if R else Q
        Q = point_addition(Q, Q, a, p)
        k >>= 1

    return R

In [ ]:
p = 67
a = 2
b = 3
e1 = (2, 22)  # Bob's public point
d = 4         # Bob's private key
P = (24, 26)  # Plaintext point Alice wants to send
r = 2         # Alice's random integer
e2 = scalar_multiplication(d, e1, a, p) #e2 = d * e1

# Step 1: Alice's encryption
C1 = scalar_multiplication(r, e1, a, p)  # C1 = r * e1
r_e2 = scalar_multiplication(r, e2, a, p)  # r * e2
C2 = point_addition(P, r_e2, a, p)    # C2 = P + r * e2

print("Ciphertext (C1, C2):", C1, C2)

# Step 2: Bob's decryption
d_C1 = scalar_multiplication(d, C1, a, p)  # d * C1
P_decrypted = point_addition(C2, (d_C1[0], -d_C1[1] % p), a, p) # C2 - (d*c1)

print("Decrypted plaintext P:", P_decrypted)


Ciphertext (C1, C2): (35, 1) (21, 44)
Decrypted plaintext P: (24, 26)


In [ ]:
print((691**347) % 941)

470


In [ ]:
print((390**781)%941)

470


In [ ]:
print(627**347 %941)

390
